In [1]:
import os
import pandas as pd
import swat
import sys
import fnmatch

## CONNECTION TO Cloud Analytic Services and activation of necessary CAS Actions sets 

In [2]:
cashost='frasepviya35.aws.sas.com'
casport=5570

sess = swat.CAS(cashost, casport)
sess.loadactionset('fedsql')
sess.loadactionset('datastep')

NOTE: Added action set 'fedsql'.
NOTE: Added action set 'datastep'.


[actionset]

 'datastep'

+ Elapsed: 0.000277s, sys: 0.000241s, mem: 0.221mb

In [3]:
%%time
castraffic=sess.CASTable(name="traffic_violations",caslib="sampledata")
sess.loadtable(path='traffic_violations.csv', caslib="sampledata", casout={"caslib":"sampledata","name":"traffic_violations", "replace":True})
cas_agg=castraffic.groupby(by=["Driver State","Arrest Type"]).summary(subset=['Min', 'Max']).concat_bygroups()

NOTE: Cloud Analytic Services made the DNFS file traffic_violations.csv available as table TRAFFIC_VIOLATIONS in caslib sampledata.
CPU times: user 3.79 s, sys: 65 ms, total: 3.86 s
Wall time: 8.97 s


In [5]:
%%time
pdtraffic=pd.read_csv("/opt/demo/sampledata/traffic_violations.csv")
pdtraffic_agg=pdtraffic.groupby(by=["Driver State","Arrest Type"]).agg(['min', 'max'])

<string>:2: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 52.8 s, sys: 3.74 s, total: 56.5 s
Wall time: 32.6 s


In [6]:
pdtraffic.corr()

,Latitude,Longitude,Year,Contributed To Accident
Latitude,1.000000,-0.999932,0.005958,-0.026601
Longitude,-0.999932,1.000000,-0.005957,0.026556
Year,0.005958,-0.005957,1.000000,-0.003318
Contributed To Accident,-0.026601,0.026556,-0.003318,1.000000


CASTable('traffic_violations', caslib='sampledata')
Data columns (total 43 columns):
                               N   Miss     Type
SeqID                    1605590  False  varchar
Date Of Stop             1605590  False  varchar
Time Of Stop             1605590  False  varchar
Agency                   1605590  False  varchar
SubAgency                1605590  False  varchar
Description              1605582   True  varchar
Location                 1605588   True  varchar
Latitude                 1605590  False   double
Longitude                1605590  False   double
Accident                 1605590  False  varchar
Belts                    1605590  False  varchar
Personal Injury          1605590  False  varchar
Property Damage          1605590  False  varchar
Fatal                    1605590  False  varchar
Commercial License       1605590  False  varchar
HAZMAT                   1605590  False  varchar
Commercial Vehicle       1605590  False  varchar
Alcohol                  1605590 

In [10]:
ops_agg = castraffic.groupby('Driver State').agg(['count'])

raw_nlargest = castraffic.nlargest(10,'Driver State')

AttributeError: agg

## STEP 0 : Get the list of filling data files located in a given directory

In [53]:
# Benchmark configurations
metric_names = ['cpu_utilization', 'latency', 'packet_loss', 'throughput']
nlargest = 10

## STEP 1 corrected : Define function in charge of importing filling data with right data types

In [57]:
%%timeit

benchmark_file = '/opt/demo/sampledata/traffic_violations.csv'
pdf = pd.read_csv(benchmark_file)

/home/ec2-user/anaconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py:1162: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types. Specify dtype option on import or set low_memory=False.
  all_runs = timer.repeat(repeat, number)


13 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
pdf.info()

NameError: name 'pdf' is not defined

In [55]:
pdf_agg = pdf.groupby(['company']).agg({k:['min','max','mean'] for k in metric_names})

/home/ec2-user/anaconda3/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


KeyError: 'company'

In [ ]:
# lets put the data into a pandas df 
# clicking on raw_nyc_phil.json under "Input Files" 
# tells us parent node is 'programs' 
nycphil = json_normalize(d['programs']) 
nycphil.head(3) 

In [ ]:
pdf.head()

In [ ]:
ops.info()

## STEP 2 : Define function in charge of appending a table to another in CAS

In [ ]:
def append_sensor_data(targetTable, incrementTable):
    appendcode="data '"+targetTable+"'(APPEND=YES); set '"+incrementTable+"'; run;"
    sess.dataStep.runCode(appendcode)

In [ ]:
for i in range(1,len(csvfilelist)):
    sess.droptable(table="TMP_LHA", quiet="true")
    import_filling_data(csvfilelist[i], "TMP_LHA")
    append_filling_data("LHA_FULL", "TMP_LHA")

In [ ]:
%%timeit

casdata = sess.CASTable()

ops.groupby(['company']).groups.keys()

In [ ]:
%%timeit

ops_agg = ops.groupby(['company']).agg({k:['min','max','mean'] for k in metric_names})

In [ ]:
raw_nlargest = ops.nlargest(nlargest, 'cpu_utilization')

In [ ]:
ops.info()

In [ ]:
ops2 = ops.groupby(by="company")

In [ ]:
ops2.head(100)

In [7]:
sess.terminate()